In [1]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import concurrent.futures
from tqdm import tqdm  # 可选进度条

In [ ]:
import pandas as pd
import numpy as np
import glob
import re
import os

# ==== 输入输出路径 ====
input_dir = "../Data_Output/Train/"
output_dir = "../Data_Preprocessing/Image/Bloch_STT_1e-3_100nm/"
os.makedirs(output_dir, exist_ok=True)
csv_files = glob.glob(os.path.join(input_dir, "bloch_STT_*_sweep*.csv"))

# ==== 帮助函数 ====
def get_key_times(t_array):
    t_start = t_array[0]
    t_end = t_array[-1]
    t_middle = t_array[np.abs(t_array - (t_start + t_end)/2).argmin()]
    return [t_start, t_middle, t_end]

# ==== 主循环 ====
for file_path in csv_files:
    print(f"正在处理文件: {file_path}")
    match = re.search(r'bloch_STT_(\d+e\d+)_sweep\d+\.csv', file_path)
    if not match:
        print(f"跳过无效文件名: {file_path}")
        continue

    jx_str = match.group(1)
    jx_val = float(jx_str)

    df = pd.read_csv(file_path, skiprows=8)
    t_array = np.sort(df["t"].unique())
    if len(t_array) < 3:
        continue

    key_times = get_key_times(t_array)
    num_cols = df.shape[1]
    jx_frames = []

    for t_val in key_times:
        base_frame = df[df["t"] == t_val][["% X", "Y", "t"]].copy()

        for col in range(3, num_cols, 5):
            mX_col = df.columns[col]
            mY_col = df.columns[col + 1]
            mZ_col = df.columns[col + 2]
            alpha_col = df.columns[col + 3]
            beta_col = df.columns[col + 4]

            alpha_val = df[alpha_col].iloc[0]
            beta_val = df[beta_col].iloc[0]

            suffix = f"@ alpha={alpha_val}; beta={beta_val}"

            base_frame[f"mX (1) {suffix}"] = df[df["t"] == t_val][mX_col].values
            base_frame[f"mY (1) {suffix}"] = df[df["t"] == t_val][mY_col].values
            base_frame[f"mZ (1) {suffix}"] = df[df["t"] == t_val][mZ_col].values

        jx_frames.append(base_frame)

    if jx_frames:
        jx_df = pd.concat(jx_frames, axis=0)
        output_path = os.path.join(output_dir, f"skx_3frames_jx={jx_str}.csv")
        jx_df.to_csv(output_path, index=False)
        print(f"✅ 已保存: {output_path}（{len(jx_df)} 行）")


In [ ]:
import pandas as pd
import numpy as np
import glob
import re
import os

# ==== 路径配置 ====
input_dir = "../Data_Output/Train/"
output_dir = "../Data_Preprocessing/Preprocessed_Dataset/jx_separated/"
os.makedirs(output_dir, exist_ok=True)

csv_files = glob.glob(os.path.join(input_dir, "bloch_STT_*_sweep*.csv"))

# ==== 获取关键时间帧 ====
def get_key_times(t_array):
    t_start = t_array[0]
    t_end = t_array[-1]
    t_middle = t_array[np.abs(t_array - (t_start + t_end) / 2).argmin()]
    return [t_start, t_middle, t_end]

# ==== 主循环 ====
for file_path in csv_files:
    match = re.search(r'bloch_STT_(\d+e\d+)_sweep\d+\.csv', file_path)
    if not match:
        print(f"跳过无效文件名: {file_path}")
        continue

    jx_str = match.group(1)
    jx_val = float(jx_str)

    print(f"\n▶️ 正在处理文件: {file_path}")
    df = pd.read_csv(file_path, skiprows=8)
    t_array = np.sort(df["t"].unique())
    if len(t_array) < 3:
        print(f"⏭️ 跳过：时间帧不足3个")
        continue

    key_times = get_key_times(t_array)
    num_cols = df.shape[1]
    jx_frames = []

    for t_val in key_times:
        df_frame = df[df["t"] == t_val]
        if df_frame.empty:
            continue

        base = df_frame[["% X", "Y", "t"]].reset_index(drop=True)
        all_magnetics = []

        for col in range(3, num_cols, 5):
            mX_col = df.columns[col]
            mY_col = df.columns[col + 1]
            mZ_col = df.columns[col + 2]
            alpha_col = df.columns[col + 3]
            beta_col = df.columns[col + 4]

            alpha_val = df[alpha_col].iloc[0]
            beta_val = df[beta_col].iloc[0]

            suffix = f"@ alpha={alpha_val}; beta={beta_val}"

            # 一次性添加列（避免 insert 多次触发碎片化）
            group = pd.DataFrame({
                f"mX (1) {suffix}": df_frame[mX_col].values,
                f"mY (1) {suffix}": df_frame[mY_col].values,
                f"mZ (1) {suffix}": df_frame[mZ_col].values
            })

            all_magnetics.append(group)

        # 合并 base + 所有磁化向量列
        frame_all = pd.concat([base] + all_magnetics, axis=1)
        jx_frames.append(frame_all)

    # 保存每个 jx 为单独 CSV
    if jx_frames:
        jx_df = pd.concat(jx_frames, axis=0)
        output_path = os.path.join(output_dir, f"skx_3frames_jx={jx_str}.csv")
        jx_df.to_csv(output_path, index=False)
        print(f"✅ 已保存到: {output_path}，共 {len(jx_df)} 行")
    else:
        print(f"⚠️ 没有有效帧可写入：{file_path}")
